In [1]:
from pathlib import Path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [3]:
!nvidia-smi

Thu Aug 14 20:05:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   43C    P8             10W /  220W |     954MiB /  12282MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0

device = torch.device(f"cuda:{DEVICE_NUM}" if torch.cuda.is_available() else "cpu")
print("INFO: Using device -", device)

INFO: Using device - cuda:0


In [2]:
from typing import Callable, Optional
from torchvision.datasets.utils import download_and_extract_archive

torchvision.datasets.utils.tqdm = tqdm


class ImageDataset(torchvision.datasets.ImageFolder):
    DOWNLOAD_URL = "https://daiv-cnu.duckdns.org/contest/ai_competition[2025]/dataset/datasets.zip"
    ARCHIVE_FILENAME = "datasets.zip"

    def __init__(
        self,
        root: Path,
        force_download: bool = True,
        train: bool = True,
        valid: bool = False,
        unlabeled: bool = False,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None
    ):
        self.download(root, force=force_download)

        if not train:
            root = root / "test"
        elif valid:
            root = root / "val"
        elif unlabeled:
            root = root / "unlabeled"
        else:
            root = root / "train"

        super().__init__(root=root, transform=transform, target_transform=target_transform)

    @classmethod
    def download(cls, root: Path, force: bool = False):
        zip_path = root / cls.ARCHIVE_FILENAME

        if force or not zip_path.exists():
            download_and_extract_archive(
                cls.DOWNLOAD_URL,
                download_root=root,
                extract_root=root,
                filename=cls.ARCHIVE_FILENAME
            )

            # Arrange Dataset Directory
            for target_dir in [root / "test", root / "unlabeled"]:
                for file_path in target_dir.iterdir():
                    if file_path.is_file() and file_path.suffix == ".JPEG":
                        new_dir = target_dir / file_path.stem
                        new_dir.mkdir(exist_ok=True)
                        file_path.rename(new_dir / file_path.name)

            print("INFO: Dataset archive downloaded and extracted.")
        else:
            print("INFO: Dataset archive found in the root directory. Skipping download.")

In [3]:
# Image Resizing and Tensor Conversion
IMG_SIZE = (64, 64)

# ImageNet Normalization
IMG_NORM = dict(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

resizer = transforms.Compose([
    transforms.Resize(IMG_SIZE), # Resize Image
    transforms.ToTensor(), # Convert Image to Tensor
    transforms.Normalize(**IMG_NORM) # Normalization
])

train_resizer = transforms.Compose([
    transforms.RandomHorizontalFlip(), # Random Horizontal Flip
    transforms.RandomVerticalFlip(), # Random Vertical Flip
    transforms.RandomRotation(10), # Random Rotation
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
    ], p=0.5),
    resizer # Resize and Normalize
])

valid_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(**IMG_NORM)
])

In [4]:
DATA_ROOT = Path("data")

train_dataset = ImageDataset(root=DATA_ROOT, force_download=False, train=True, transform=train_resizer)
valid_dataset = ImageDataset(root=DATA_ROOT, force_download=False, valid=True, transform=valid_transform)
test_dataset = ImageDataset(root=DATA_ROOT, force_download=False, train=False, transform=resizer)
unlabeled_dataset = ImageDataset(root=DATA_ROOT, force_download=False, unlabeled=True, transform=resizer)

print(f"INFO: Dataset loaded successfully. Number of samples - Train({len(train_dataset)}), Valid({len(valid_dataset)}), Test({len(test_dataset)}), Unlabeled({len(unlabeled_dataset)})")

INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Dataset loaded successfully. Number of samples - Train(8000), Valid(10000), Test(19996), Unlabeled(72004)


In [5]:
print(f"INFO: Train dataset has been overridden with augmented state. Number of samples - Train({len(train_dataset)})")

INFO: Train dataset has been overridden with augmented state. Number of samples - Train(8000)
